# CGM Data Quality Analysis Tutorial
## Using cgm-data-processor with XDrip+ Backups

This notebook demonstrates a practical workflow for processing Continuous Glucose Monitoring (CGM) data using the cgm-data-processor tool. We'll walk through loading data from an XDrip+ SQLite backup, performing quality assessments, and exporting the processed data in a standardized format suitable for further analysis.

### Overview

The cgm-data-processor tool simplifies the process of working with CGM data by handling common preprocessing tasks and standardizing the output format. This example focuses on three key aspects:

1. Data Loading: Extracting CGM measurements, carbohydrate records, and insulin data from an XDrip+ SQLite backup
2. Quality Assessment: Evaluating data completeness, identifying gaps, and assessing measurement reliability
3. Standardized Export: Saving the processed data in a consistent CSV format that facilitates further analysis

### Prerequisites

Before running this notebook, ensure you have:
- The cgm-data-processor package installed
- An XDrip+ SQLite backup file
- Basic familiarity with Python and pandas

### Expected Output

The processed dataset will include:
- Glucose measurements aligned to 5-minute intervals
- Values in both mg/dL and mmol/L units
- Validated carbohydrate and insulin records
- Quality metrics for each time period
- Clearly marked data gaps and interpolated values

### Data Quality Considerations

Throughout this tutorial, we'll examine several key quality metrics:
- Measurement frequency and consistency
- Gap duration and distribution
- Sensor reliability indicators
- Record completeness for insulin and carbohydrate data

This quality assessment helps ensure that subsequent analyses are based on reliable data and that any limitations are well understood.

### Next Steps

After completing this tutorial, you'll have a standardized dataset ready for various analyses such as:
- Glucose variability assessment
- Meal response patterns
- Insulin sensitivity calculations
- Time-in-range analysis

Let's begin by importing the necessary libraries and setting up our environment.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:95% !important; }</style>")) # Make Jupyter cells wider for better visuals
import pprint

In [4]:
# Path modification used to allow Notebook access to src directory
import os
import sys
notebook_path = os.path.abspath('.')
project_root = os.path.join(notebook_path, '../../')
if project_root not in sys.path:
    sys.path.append(project_root)

# Preprocessing Module - Load, clean and align data
from src.preprocessing.loading import XDrip
from src.preprocessing.cleaning import clean_classify_insulin, clean_classify_carbs, clean_glucose
from src.preprocessing.alignment import align_diabetes_data

# Analysis Module - Check and display data quality
from src.analysis.gaps import analyse_glucose_gaps
from src.analysis.insulin import analyse_insulin_over_time
from src.analysis.metrics import display_quality_metrics

# Visualisation Module - Format data for visual appeal in Jupyter
from src.visualisation.quality_dashboard import create_quality_dashboard
from src.visualisation.meal_statistics_dashboard import create_meal_statistics_dashboard
from src.visualisation.gap_dashboard import create_gap_dashboard